In [ ]:
#TODO: merge file with NCBI parsed?

In [1]:
import sys , os 
import urllib, pickle


In [53]:
# LOAD from pickle
with open('/home/antonis/data/biocreative6/pubmed_ids_extended.pickle', 'rb') as f:
    pubmed_ids = pickle.load(f)

In [3]:
len(pubmed_ids['development']+pubmed_ids['training']+pubmed_ids['test_gs'])

2432

In [4]:
#define from which ids to take citations from
ids = pubmed_ids['development']+pubmed_ids['training']+pubmed_ids['test_gs']
print 'Will search for citations from %i PubMed articles'%len(ids)

Will search for citations from 2432 PubMed articles


In [5]:
bioconcept = 'Bioconcept' # includes all formats (genes, chemicals, diseases...)
api_output_loc = 'temp/citations_list.txt'

In [59]:
# TODO: move to pubmed_utils.py or download_data.py 
# n=199 #split to chunks
# cites = True # 
def get_pubmed_citations(ids, outgoing = True, api_output_loc='temp/citations_list.txt', chunks=199):
    """Get the references/citations of a list of pubmed ids.
    
    ids: list of articles to get citations/refs from
    outgoing: If True, return the documents that the given article(s) cite (outgoing citations),
            otherwise return documents that have cited that article (incoming citations).
    chunks: how many articles to request at once
    api_output_loc: location to save temp txt file
    
    """
    citations = []
    for i in xrange(0,len(ids), n):
        print 'Download citations from articles %i to %i  . . . .'%(i+1,i+n+1)
        pmids = ','.join(ids[i:i+n])
        if outgoing:
            url_Submit = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_refs&id=%s&tool=my_tool&email=my_email@example.com"%(pmids)
        else:
            url_Submit = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id=%s&tool=my_tool&email=my_email@example.com"%(pmids)
        urllib.urlretrieve(url_Submit, api_output_loc)
    #     print url_Submit
        tree = ET.parse(api_output_loc)
        root = tree.getroot()
        citations.extend(map(lambda x: x.text ,root.findall("./LinkSet/LinkSetDb/Link/Id")))
        
    print "Found %s citations (%s unique) from %s articles"%(len(citations),len(set(citations)), len(ids))
    return citations

In [60]:
pubmed_ids['outgoing_citations'] = get_pubmed_citations(ids,outgoing = True)
pubmed_ids['incoming_citations'] = get_pubmed_citations(ids,outgoing = False)


Download citations from articles 1 to 200  . . . .
Download citations from articles 200 to 399  . . . .
Download citations from articles 399 to 598  . . . .
Download citations from articles 598 to 797  . . . .
Download citations from articles 797 to 996  . . . .
Download citations from articles 996 to 1195  . . . .
Download citations from articles 1195 to 1394  . . . .
Download citations from articles 1394 to 1593  . . . .
Download citations from articles 1593 to 1792  . . . .
Download citations from articles 1792 to 1991  . . . .
Download citations from articles 1991 to 2190  . . . .
Download citations from articles 2190 to 2389  . . . .
Download citations from articles 2389 to 2588  . . . .
Found 22973 citations (22291 unique) from 2432 articles
Download citations from articles 1 to 200  . . . .
Download citations from articles 200 to 399  . . . .
Download citations from articles 399 to 598  . . . .
Download citations from articles 598 to 797  . . . .
Download citations from articles

In [62]:
pubmed_ids.keys()

['development',
 'incoming_citations',
 'training',
 'test_gs',
 'outgoing_citations',
 'chemdner_silver']

### delete from citations documents contained on train,dev, test set

In [15]:
print '# outgoing_citations:',len(pubmed_ids['outgoing_citations'])
pubmed_ids['outgoing_citations'] = list(set(pubmed_ids['outgoing_citations']) - set(pubmed_ids['development'] + pubmed_ids['training'] + pubmed_ids['test_gs']))
print '# outgoing_citations without overlap with traintestdev:',len(pubmed_ids['outgoing_citations'])

print '# incoming_citations:',len(pubmed_ids['incoming_citations'])
pubmed_ids['incoming_citations'] = list(set(pubmed_ids['incoming_citations']) - set(pubmed_ids['development'] + pubmed_ids['training'] + pubmed_ids['test_gs']))
print '# incoming_citations without overlap with traintestdev:',len(pubmed_ids['incoming_citations'])


# outgoing_citations: 22973
# outgoing_citations without overlap with traintestdev: 22251
# incoming_citations: 29059
# incoming_citations without overlap with traintestdev: 27546


In [16]:
with open('/home/antonis/data/biocreative6/pubmed_ids_citations.pickle', 'wb') as f:
    pickle.dump(pubmed_ids, f)

In [64]:
# Check some of the articles

In [65]:
# import random
# # get article link
# print 'https://www.ncbi.nlm.nih.gov/pubmed/?term='+citations[random.randint(0,len(citations))]
# # get article entities
# print "https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/RESTful/tmTool.cgi/Bioconcept/%s/PubTator/"%citations[random.randint(0,len(citations))]
